
discord music player?

export TOKENPATH=/Users/briancullinan/.credentials/discord-bot7.txt && export DEFAULT_APPLICATION=1335769252409380884 && export SECRETPATH=/Users/briancullinan/.credentials/discord7.txt && npm run import -- "discord music player"


In [ ]:

const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const { startServer, closeExpress } = importer.import('express service')
const {doMention, doPrivate} = importer.import('discord llm interactions')

const ALL_COMMANDS = [
  'browse',
]

// bot commands using new API, same names as above but lower-case
async function playerCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  interactionsCommands['startActivity'] = startServer.bind(null, ['discord player.ipynb'])
  interactionsCommands['endActivity'] = closeExpress

  interactionsCommands['promptMention'] = doMention
  interactionsCommands['promptPrivate'] = doPrivate

  cmdDef = {
    'name': 'browse',
    'description': 'Launch the music player',
    'type': 4,
    'handler': 2,
  }
  if(!commands.includes('browse')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'browse')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }

  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = playerCommands





discord music player server?

ROUTE = /player

ROOT = true

AUTHENTICATE = importer.import('discord authenticate instances').authenticateRoute



In [ ]:
const {BASE_DIRECTORY} = importer.import('node express directory handler')
const {DEFAULT_APPLICATION} = importer.import('discord bot configuration')
const path = require('path')
const directoryToHtml = importer.import('directory to html')

async function discordLogin(baseURI, req, res, next) {
  let basePath = path.resolve(BASE_DIRECTORY)

  let combined = path.join(basePath, decodeURIComponent(req.path))
  console.log(combined)

  if(!fs.existsSync(combined)) {
    return next()
  }

  if(!fs.statSync(combined).isDirectory()) {
    return next()
  }

  // this is exclusively a discord function so it has to be appended somewhere before using the generic directory lister
  let htmlCode = await directoryToHtml(combined, req.path.split('/').length > 2, req.authenticated)
  htmlCode = htmlCode.replace('<head>', `<head>
    <base href="/.proxy/" />
    <script type="text/javascript">
    ${importer.interpret('discord client auth code').code
      .replaceAll('{CLIENT_ID}', DEFAULT_APPLICATION)
      .replaceAll('{BASE_URI}', baseURI)}
    </script>`)
  return res.send(htmlCode)
}

module.exports = discordLogin
